In [2]:
!pip install sentence-transformers faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 61.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 95.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 86.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

# Семантический поиск

## Пример лексического поиска по словам

In [1]:
import json
import string
from collections import defaultdict

# Загрузка данных
def load_data(filename):
    with open(filename, 'r', encoding='utf-8') as f:
        return json.load(f)

# Подготовка текста
def preprocess_text(text):
    text = text.lower().translate(str.maketrans('', '', string.punctuation))
    russian_stopwords = {'и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а',
                        'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же',
                        'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от',
                        'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже',
                        'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него',
                        'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом',
                        'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо',
                        'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без',
                        'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда',
                        'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним',
                        'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее',
                        'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при',
                        'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше',
                        'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много',
                        'разве', 'три', 'эту', 'моя', 'впрочем', 'хорошо', 'свою', 'этой',
                        'перед', 'иногда', 'лучше', 'чуть', 'том', 'нельзя', 'такой', 'им',
                        'более', 'всегда', 'конечно', 'всю', 'между'}
    return [word for word in text.split() if word not in russian_stopwords]

# Построение поискового индекса
def build_search_index(data):
    index = defaultdict(list)
    for idx, item in enumerate(data['data']):
        words = preprocess_text(item['question'])
        for word in set(words):
            index[word].append(idx)
    return index

# Функция поиска
def search(query, data, index, top_n=5):
    query_words = preprocess_text(query)
    scores = defaultdict(int)

    for word in query_words:
        if word in index:
            for doc_id in index[word]:
                scores[doc_id] += 1

    sorted_results = sorted(scores.items(), key=lambda x: x[1], reverse=True)

    results = []
    for doc_id, score in sorted_results[:top_n]:
        item = data['data'][doc_id]
        processed_question = preprocess_text(item['question'])
        match_terms = set(query_words) & set(processed_question)
        results.append({
            'question': item['question'],
            'answer': item['answer'],
            'category': item['category'],
            'score': score,
            'match_terms': list(match_terms)  # Исправлено: добавлены недостающие скобки
        })

    return results

# Основная программа
if __name__ == "__main__":
    data = load_data('qa_data.json')
    search_index = build_search_index(data)

    query = "Как изменить график работы?"
    results = search(query, data, search_index)

    print(f"Результаты поиска для: '{query}'\n")
    for i, result in enumerate(results, 1):
        print(f"{i}. [{result['category']}] {result['question']}")
        print(f"   Ответ: {result['answer']}")
        print(f"   Совпадения: {result['score']} ({', '.join(result['match_terms'])})")
        print("-" * 80)

Результаты поиска для: 'Как изменить график работы?'

1. [график работы] как изменить процент занятости?
   Ответ: Для изменения процента занятости сотруднику создайте, пожалуйста, заявку на сотрудника по теме "Изменение режима, характера работы"
   Совпадения: 1 (изменить)
--------------------------------------------------------------------------------


## Семантический поиск

In [4]:
import json
import string
import numpy as np
from sentence_transformers import SentenceTransformer
import faiss

In [5]:
# Загрузка данных
def load_data(filename):
    with open(filename, 'r', encoding='utf-8') as f:
        return json.load(f)

In [6]:
# Предобработка текста (опционально, можно убрать, т.к. BERT умеет сам)
def preprocess_text(text):
    return text.lower().translate(str.maketrans('', '', string.punctuation))

def build_semantic_index(data, model):
    questions = [preprocess_text(item['question']) for item in data['data']]
    embeddings = model.encode(questions, convert_to_numpy=True)

    index = faiss.IndexFlatL2(embeddings.shape[1])
    index.add(embeddings)

    return index, embeddings

In [7]:
# Семантический поиск
def semantic_search(query, data, model, index, top_n=5):
    query_vec = model.encode([preprocess_text(query)], convert_to_numpy=True)
    distances, indices = index.search(query_vec, top_n)

    results = []
    for idx, dist in zip(indices[0], distances[0]):
        item = data['data'][idx]
        results.append({
            'question': item['question'],
            'answer': item['answer'],
            'category': item['category'],
            'score': float(dist),  # расстояние (чем меньше — тем ближе)
        })
    return results

In [8]:
if __name__ == "__main__":
    # Загрузка модели
    model = SentenceTransformer('all-MiniLM-L6-v2')

    # Загрузка данных
    data = load_data('qa_data.json')

    # Построение индекса
    search_index, embeddings = build_semantic_index(data, model)


    query = "Как изменить график работы?"
    results = semantic_search(query, data, model, search_index)
    print(f"Результаты семантического поиска для: '{query}'\n")
    for i, result in enumerate(results, 1):
        print(f"{i}. [{result['category']}] {result['question']}")
        print(f"   Ответ: {result['answer']}")
        print(f"   Расстояние (чем меньше, тем ближе): {result['score']:.4f}")
        print("-" * 80)


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Результаты семантического поиска для: 'Как изменить график работы?'

1. [больничный] Как загрузить больничный лист?
   Ответ: На текущий момент данные по больничным листам поступают работодателю от СФР напрямую. Если больничный лист не был подтвержден, а затем появилась необходимость в его проведении, то сотруднику следует создать заявку в Личном кабинете по теме "Перерасчет пособий" и указать номер ЭЛН.
   Расстояние (чем меньше, тем ближе): 0.5773
--------------------------------------------------------------------------------
2. [график работы] как изменить процент занятости?
   Ответ: Для изменения процента занятости сотруднику создайте, пожалуйста, заявку на сотрудника по теме "Изменение режима, характера работы"
   Расстояние (чем меньше, тем ближе): 0.6348
--------------------------------------------------------------------------------
3. [Популярные фразы] вернуть старый дизайн
   Ответ: Чат-бот находится в стадии пилотирования и обучается ежедневно. Пожалуйста, обратитесь в по